In [ ]:
import json
import time
import google.generativeai as genai

genai.configure(api_key="YOUR_TOKEN_KEY")
model = genai.GenerativeModel('gemini-2.0-flash-001')

In [ ]:
with open("extracted_mmqa.json", "r", encoding="utf-8") as f:
    mmqa_data = json.load(f)


In [ ]:
def answer_prompt(context: str, question: str) -> str:
    return f"""Context:
{context}

Question:
{question}

Please provide a concise answer in exactly one sentence, without any further explanation or calculation steps. It should start with "Answer: "

Answer:"""

def evidence_prompt(context: str, question: str) -> str:
    return f"""In your evidence you should:
- Explain in full English why each piece of data supports your answer. Strictly start it with "Evidence:"

Context:
{context}

Question:
{question}

Answer and Evidence:"""

In [ ]:
def generate_with_retries(prompt: str, max_retries: int = 5, initial_delay: int = 5):
    delay = initial_delay
    for attempt in range(1, max_retries + 1):
        try:
            return model.generate_content(prompt)
        except Exception as e:
            if "429" in str(e) or "quota" in str(e).lower():
                print(f"[Rate Limit] Attempt {attempt}: retrying in {delay}s…")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[Error] {e}")
                raise
    raise RuntimeError("Max retries exceeded")


In [ ]:
def run_batch(start_idx, end_idx):
    results = []
    print(f"\n🚀 Running batch: {start_idx + 1} to {min(end_idx, len(mmqa_data))}")

    for i in range(start_idx, min(end_idx, len(mmqa_data))):
        q = mmqa_data[i]
        print(f"🔍 Processing #{i + 1}: UID = {q['qid']}")

        try:
            response = generate_with_retries(answer_prompt(q["context"], q["question"]))
            ans = response.text.strip()
        except Exception as e:
            print(f"[Answer Error] UID={q['qid']}: {e}")
            ans = ""

        try:
            response_ev = generate_with_retries(evidence_prompt(q["context"], q["question"]))
            ev_text = response_ev.text
        except Exception as e:
            print(f"[Evidence Error] UID={q['qid']}: {e}")
            ev_text = ""

        snippet = ""
        if "Evidence:" in ev_text:
            snippet = ev_text.split("Evidence:")[-1]
        elif "The gold evidence is:" in ev_text:
            snippet = ev_text.split("The gold evidence is:")[-1]

        lines = [line.strip() for line in snippet.splitlines() if line.strip()]
        gold_evidence = "\n".join(lines) if lines else "Not Available"

        print(f"Q: {q['question']}")
        print(f"✔️ Answer: {ans}")
        print(f"📚 Evidence: {gold_evidence}")
        print("-" * 60)

        results.append({
            "qid": q["qid"],
            "question": q["question"],
            "ans": ans,
            "gold_evidence": gold_evidence,
            "gold_answer": q["answer"]
        })

        time.sleep(0.5)

    filename = f"mmqa_batch_{start_idx + 1}_{min(end_idx, len(mmqa_data))}.json"
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)
    print(f"✅ Batch saved: {filename}")

In [ ]:
batch_size = 500
start_from = 4000
end_at = 5000

for start in range(start_from, end_at, batch_size):
    end = start + batch_size
    run_batch(start, end)


🚀 Running batch: 4001 to 4500
🔍 Processing #4001: UID = 828d1066d75bf6ba13952e1760f40237
Q: what is the difference between acc and ncaa
✔️ Answer: Answer: The NCAA is a larger national governing body for collegiate athletics, while the ACC is a specific athletic conference within the NCAA.
📚 Evidence: The text states, "The Atlantic Coast Conference (ACC) is a collegiate athletic conference in the United States of America in which its fifteen member universities compete in the National Collegiate Athletic Association (NCAA)'s Division I." This clearly indicates that the ACC is a conference within the larger NCAA organization.
------------------------------------------------------------
🔍 Processing #4002: UID = c1e4fb1fb9f9937aafe0cbe220dfefcb
Q: Somos tú y yo is a Venezuelan telenovela filmed in Caracas, Venezuela and developed by which cable and satellite television channel owned by Turner Broadcasting System, a unit of Time Warner and its main flagship channel of Cartoon Network Asi